In [ ]:
import sys
sys.path.append('..')
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '2'
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
from scipy.stats import beta
import warnings
from src.lesion import perform_lesion_experiment, do_lesion_hypo_tests
from src.pointers import DATA_PATHS
from src.experiment_tagging import get_model_path
from src.utils import bates_quantile
from scipy.stats import sem

In [ ]:
models = [# 'CNN-VGG-CIFAR10',
          'CNN-VGG-CIFAR10+DROPOUT+L2REG',
          # 'CNN-VGG-CIFAR10+L1REG', 'CNN-VGG-CIFAR10+L2REG',
          # 'CNN-VGG-CIFAR10+DROPOUT', , 'CNN-VGG-CIFAR10+MOD-INIT'
          ]

n_clusts = (8, 12)
n_shuffles = 19
n_workers = 5
n_reps = 5
is_unpruned = True
dataset_name = 'cifar10_full'
# os.environ['CUDA_VISIBLE_DEVICES'] = '1,3'
results_dir = '/project/nn_clustering/results/'

In [ ]:
all_results = []

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for tag in tqdm(models):
        for n_clust in n_clusts:
            for use_activations in [False, True]:
                for do_local in [False, True]:
                    paths = get_model_path(tag, filter_='all')[-n_reps:]

                    net_pkl_results = []
                    fisher_p_means, chi2_p_means, effect_means = [], [], []
                    fisher_p_ranges, chi2_p_ranges, effect_ranges = [], [], []
                    fisher_stat_means, fisher_stat_ranges = [], []

                    for path in paths:

                        (true_results,
                         all_random_results,
                         metadata,
                         evaluation) = perform_lesion_experiment('.' + DATA_PATHS[dataset_name],
                                                                 path, n_clusters=n_clust,
                                                                 n_shuffles=n_shuffles,
                                                                 unpruned=is_unpruned,
                                                                 depth=3,
                                                                 n_side=32,
                                                                 activations=use_activations,
                                                                 local=do_local)
                        net_pkl_results.append({'true_results': true_results,
                                                'all_random_results': all_random_results,
                                                'metadata': metadata,
                                                'evaluation': evaluation})
                        hypo_results = do_lesion_hypo_tests(evaluation, true_results, all_random_results)

                        fisher_p_means.append(hypo_results['fisher_p_means'])
                        fisher_stat_means.append(hypo_results['fisher_stat_means'])
                        chi2_p_means.append(hypo_results['chi2_p_means'])
                        effect_means.append(hypo_results['effect_factors_means'])
                        fisher_p_ranges.append(hypo_results['fisher_p_ranges'])
                        fisher_stat_ranges.append(hypo_results['fisher_stat_ranges'])
                        chi2_p_ranges.append(hypo_results['chi2_p_ranges'])
                        effect_ranges.append(hypo_results['effect_factors_range'])
                    print(tag, use_activations, do_local)

                    with open(results_dir + '/lesion_data_' + tag + f'_activations={use_activations}_local={do_local}_k={n_clust}.pkl', 'wb') as f:
                        pickle.dump(net_pkl_results, f)
